# 🧬 Solution_RD: Genetic Algorithm Individual for Resource Distribution

This notebook documents the `Solution_RD` class, a domain-specific implementation of the `Individual` abstract base class tailored for solving a resource distribution problem involving participants, houses, and courses. The objective balances logistical and social criteria using a flexible and pluggable design.

## 🧱 Class Overview

- `Solution_RD`: A concrete subclass of `Individual` that represents a candidate solution for assigning participants to houses and courses.
  - Representation encodes both **house allocation** and **course enrollment**.
  - Fitness balances two components:
    - **Social fitness**: Encourages participant mixing across courses.
    - **Logistic fitness**: Penalizes long distances between participant houses (based on a distance matrix).
  - Customizable fitness trade-off via `alpha` and `beta`.

## ⚙️ Parameters

- `data_matrix`: A 2D numpy array of distances between participants.
- `account_for_previous_fitness`: If `True`, tracks fitness changes over time.
- `probability_social_mutation`: Probability of performing a social mutation.
- `alpha`, `beta`: Weights controlling the importance of social vs. logistic fitness.

## ✅ Features Demonstrated

1. **Random Initialization**
   - Genome includes both house and course assignments.
   - Ensures constraints like house capacity and number of houses are respected.

2. **Fitness Evaluation**
   - Calculated as a weighted sum of:
     - Social interaction score (maximize diversity in house-course assignments).
     - Logistic penalty (minimize travel distances).
   - Supports lazy evaluation and caching.

3. **Mutation**
   - Mutation type (social or logistic) chosen probabilistically.
   - Mutates in-place.

4. **Crossover**
   - Combines genome segments from two parents (to be implemented or overridden if necessary).

5. **Parameterization**
   - Adjust evolutionary pressure via `alpha`, `beta`, and mutation probabilities.

6. **Fitness History**
   - Tracks `previous_fitness` and `next_fitness` for advanced evolutionary strategies.

## 🧠 Design Highlights

- Extends `Individual` with problem-specific logic, maintaining modularity.
- Balances domain constraints with genetic flexibility.
- Supports experimentation with different fitness weights and mutation strategies.

This class is suitable for constrained optimization tasks in social or logistical planning domains and can be integrated seamlessly into the larger genetic algorithm framework.


# Data Ingestion

In [1]:
import os
import pandas as pd
os.chdir(os.pardir) # comment if you are running this more than once
data_matrix_df = pd.read_csv('data/distance_matrix_official.csv', index_col=0)
data_matrix_np = data_matrix_df.to_numpy()
data_matrix_np.shape

(68, 68)

# Testing Features

## Create two individuals


In [3]:
from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.genome import Genome
from Genetic_algorithm.solution_rd import SolutionRD

my_fitness = ResourceFitness(data_matrix_np)
my_genome = Genome()

first_solution = SolutionRD(my_fitness, Genome)
secomnd_solution = SolutionRD(my_fitness, Genome)

## Crossover


In [6]:
child1, child2 = first_solution @ secomnd_solution
print("Fitness (child):", float(child1))
print("Fitness (child):", float(child2))

Fitness (child): 2.619160275575198
Fitness (child): 2.6218171406757014


## Mutation


In [7]:
mutant = child1.copy_Individual()
mutant.mutation()
print("original Individual:", child1.genome)
print("Mutated Individual:", mutant.genome)
print("Fitness (original):", float(child1))
print("Fitness (mutated):", float(mutant))

original Individual: 12-1010232641-15-1970817569-13820-14401-13267985-1
Mutated Individual: 12-1010232641-15-1970817569-13820-14401-13297685-1
Fitness (original): 2.619160275575198
Fitness (mutated): 2.7191602755751982


## Iteration (generate mutated versions) (has to be resolved)

In [12]:
print("\nMultiple Mutations:")
for i, m in enumerate(child1):
    print(f"Mutation {i+1}:", m, "Fitness:", m)


Multiple Mutations:
mutating: Fitness: 2.619160275575198
Mutation 1: None Fitness: None
mutating: Fitness: 2.619160275575198
Mutation 2: None Fitness: None
mutating: Fitness: 2.619160275575198
Mutation 3: None Fitness: None
mutating: Fitness: 2.619160275575198
Mutation 4: None Fitness: None
mutating: Fitness: 2.619160275575198
Mutation 5: None Fitness: None


## Operator Overloading

In [16]:
print("\nUsing ** Operator for Self-Crossover")
clones = child1 ** 3
for i, c1, in enumerate(clones):
    print(f"Clone {i+1}:", c1, "Fitness:", c1)
   


Using ** Operator for Self-Crossover
Clone 1: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1162b4230>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x1162b40e0>) Fitness: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1162b4230>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x1162b40e0>)
Clone 2: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1162fdac0>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x106bb9fd0>) Fitness: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1162fdac0>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x106bb9fd0>)
Clone 3: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1162ad640>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x1162ac980>) Fitness: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1162ad640>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x1162ac980>)


## Comparison


In [17]:
print("\nIs Individual 1 better than Individual 2?", child1 > child2)


Is Individual 1 better than Individual 2? False


## Sorting and Deduplication via Set

In [9]:
print("\n=== Sorting Individuals by Fitness ===")
individuals = [ind1 for _ in ind1]

for i, ind in enumerate(individuals):
    print(f"Ind {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

sorted_individuals = sorted(individuals)
print("\nSorted by fitness (ascending):")
for i, ind in enumerate(sorted_individuals):
    print(f"Rank {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")


=== Sorting Individuals by Fitness ===
total distance of part partecipant3 is 49.2 at course 0
total distance of part partecipant8 is 49.2 at course 0
total distance of part partecipant5 is 49.2 at course 0
total distance of part partecipant1 is 47.1 at course 0
total distance of part partecipant0 is 49.2 at course 0
total distance of part partecipant9 is 49.4 at course 0
total distance of part partecipant7 is 49.4 at course 0
total distance of part partecipant4 is 51.5 at course 0
total distance of part partecipant2 is 49.4 at course 0
total distance of part partecipant6 is 49.4 at course 0
total distance of part partecipant8 is 33.3 at course 1
total distance of part partecipant5 is 5.3 at course 1
total distance of part partecipant3 is 33.3 at course 1
total distance of part partecipant0 is 5.3 at course 1
total distance of part partecipant4 is 33.3 at course 1
total distance of part partecipant6 is 3.1 at course 1
total distance of part partecipant1 is 3.1 at course 1
total distan

## Storing Unique Individuals in a Set

In [18]:
print("\n=== Deduplication with Set ===")
ind2 = child1.copy_Individual(delete_fitness=False)  # Exact copy

# Mutate a third individual to likely get a different genome
ind3 = child1.copy_Individual(delete_fitness=False)
ind3.mutation()

ind_set = {child1, ind2, ind3}

for i, ind in enumerate(ind_set):
    print(f"Unique {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

print(f"\nTotal unique individuals: {len(ind_set)}")


=== Deduplication with Set ===
Unique 1: Genome = 12-1010232641-15-1970817569-13820-14401-13267985-1, Fitness = 2.619160275575198

Total unique individuals: 1


# accessing the genome (removed)

In [19]:
child1[0:4]

TypeError: 'SolutionRD' object is not subscriptable

In [22]:
print("Before genome swap:", child1.genome.course_assignments)
child1.genome.swap_course_assignments(0,0, 1)
print("After genome swap:", child1.genome.course_assignments)

Before genome swap: [ 3  2  6  4  1 -1  5 -1  9  7  0  8  1  7  5  6  9 -1  3  8  2  0 -1  4
  4  0  1 -1  3  2  6  7  9  8  5 -1]
After genome swap: [ 2  3  6  4  1 -1  5 -1  9  7  0  8  1  7  5  6  9 -1  3  8  2  0 -1  4
  4  0  1 -1  3  2  6  7  9  8  5 -1]


In [23]:
try:    
    child1.genome.swap_course_assignments(0,0, 10000)
except IndexError as e:
    print("Error:", e)


Error: Index out of range for course assignments.
